In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
/FileStore/tables/sales_csv.txt


FileStore/tables/menu_csv.txt


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1615632455331945>:1
----> 1 FileStore/tables/sales_csv.txt()
      4 FileStore/tables/menu_csv.txt

NameError: name 'FileStore' is not defined

In [0]:
from pyspark.sql.types import StructType,StringType,StructField,IntegerType,StringType,DateType

schema=StructType([
    StructField("product_id",IntegerType(),True),
    StructField("customer_id",StringType(),True),
    StructField("order_date",DateType(),True),
    StructField("location",StringType(),True),
    StructField("source_order",StringType(),True)
])

sales_df=spark.read.format("csv").option("inferschema","true").schema(schema).load("/FileStore/tables/sales_csv.txt")

display(sales_df)

In [0]:
from pyspark.sql.functions import month,year,quarter

sales_df=sales_df.withColumn("order_year",year(sales_df.order_date))
sales_df=sales_df.withColumn("order_month",month(sales_df.order_date))
sales_df=sales_df.withColumn("order_quarter",quarter(sales_df.order_date))
display(sales_df)

In [0]:
from pyspark.sql.types import StructType,StringType,StructField,IntegerType,StringType,DateType

schema=StructType([
    StructField("product_id",IntegerType(),True),
    StructField("product_name",StringType(),True),
    StructField("price",StringType(),True),
])

menu_df=spark.read.format("csv").option("inferschema","true").schema(schema).load("/FileStore/tables/menu_csv.txt")

display(menu_df)


In [0]:
total_amt_spent=(sales_df.join(menu_df,"product_id")
                 .groupBy("customer_id")
                 .agg({"price":"Sum"})
                 .orderBy("customer_id"))


display(total_amt_spent)

In [0]:
total_amt_spent_onfood=(sales_df.join(menu_df,"product_id")
                        .groupBy("product_name")
                        .agg({"price":"Sum"})
                        .orderBy("product_name"))

display(total_amt_spent_onfood)

In [0]:
total_amt_sales_of_month=(sales_df.join(menu_df,"product_id")
                        .groupBy("order_month")
                        .agg({"price":"Sum"}))
                        

display(total_amt_sales_of_month)

In [0]:
total_amt_sales_of_year=(sales_df.join(menu_df,"product_id")
                        .groupBy("order_year")
                        .agg({"price":"Sum"}))

display(total_amt_sales_of_year)

In [0]:
total_amt_sales_of_quarter=(sales_df.join(menu_df,"product_id")
                        .groupBy("order_quarter")
                        .agg({"price":"Sum"}))

display(total_amt_sales_of_quarter)

In [0]:
from pyspark.sql.functions import count


most_df=(sales_df.join(menu_df,"product_id")
                        .groupBy("product_id","product_name")
                        .agg(count("product_id").alias("product_count"))
                        .orderBy('product_count',ascending=0)
                        .drop('product_id')
                        )               

display(most_df)

In [0]:
from pyspark.sql.functions import count


most_df=(sales_df.join(menu_df,"product_id")
                        .groupBy("product_id","product_name")
                        .agg(count("product_id").alias("product_count"))
                        .orderBy('product_count',ascending=0)
                        .drop('product_id')
                        .limit(1)
                        )               

display(most_df)

In [0]:
from pyspark.sql.functions import countDistinct


df=(sales_df.filter(sales_df.source_order=='Restaurant')
                        .groupBy("customer_id")
                        .agg(countDistinct("order_date")))         

display(df)

In [0]:
df=(sales_df.join(menu_df,'product_id').groupBy("location").agg({'price':'sum'}))
display(df)

In [0]:
df=(sales_df.join(menu_df,'product_id').groupBy("source_order").agg({'price':'sum'}))
display(df)